In [1]:
# index對應到例子，若是片語來的例句，補上何片語
file = open('index2exam.txt', 'w') 

# index對應到有哪些gp
file1 = open('cam(index_GP).txt', 'w')

unWantADV = {'by', 'maybe', 'evenly', 'as', 'eventually', 'elsewhere', 'soon', 'sometims', 'now', 'evetually', 'after', 'below', 'so', 'exactly', 'even', 'instead', 'merely', 'very', 'anymore', 'then', 'still', 'really', 'once', 'actually', 'ever', 'in', 'also', 'only', 'just', 'else', 'anytime', 'rather', 'indeed', 'again', 'of', 'likely', 'ago', 'further', 'all', 'likewise', 'already', 'thereby', 'together', 'enough', 'everyday', 'before', 'much', 'almost', 'altogether', 'anyway', 'always', 'everywhere', 'on', 'meanwhile', 'best', 'afterward'}

# CAM need extract GP for every sentence map to index

In [2]:
import spacy
from extractGP import *
nlp = spacy.load('en_core_web_lg')

def getRoot(dependency):
    root = [ (iword, word, lemma, dep, tag, pos, ihead, NER) for iword, word, lemma, dep, tag, pos, ihead, NER in dependency if dep == 'ROOT' ]
    if not root: return []
    else: return root[0][0]

def cleanCambridgeSent(sent):
    sent = re.sub('\[ [\w +-]*\]', '', sent)
    for bracket in re.findall('(?P<word>\([\w ]*\))', sent):
        if bracket[1:3] in ['US', 'UK']: 
            sub_ = ' \\' + bracket[:-1] + '\\' + bracket[-1]
            sent = re.sub(sub_, '', sent)
        else:
            sub_ = '\\' + bracket[:-1] + '\\' + bracket[-1]
            sent = re.sub(sub_, bracket[1:-1], sent)
    sent = re.sub(' \(=.*\)', '', sent)

    for f, b in re.findall('(?P<f>\w+)'+'/'+'(?P<b>\w+)', sent):
        sent = re.sub('/'+b, '', sent)
    return sent.strip()

def sentence_to_dependencyNER(sent):
    dep = []
    doc = nlp(sent)

    POSMapping = defaultdict(lambda: '')
    for token in doc:
        POSMapping[token.text] = token.pos_

    NERMapping = defaultdict(lambda: ' ')
    # by spacy
    for ent in doc.ents:
        if ent.label_ in {'NORP', 'GPE', 'LANGUAGE', 'DATE', 'TIME', 'MONEY'}:
            texts = [ word for word in ent.text.split() if POSMapping[word] in ['NUM', 'NOUN', 'PROPN'] ]
            if texts:
                if ent.label_ == 'DATE': NERMapping[texts[-1]] = 'TIME'
                else: NERMapping[texts[-1]] = ent.label_
    # by word itself
    for token in doc:
        upperWord = token.text.upper()
        if upperWord in {'LANGUAGE', 'DATE', 'TIME', 'MONEY'} and NERMapping[token.text] == ' ':
            NERMapping[token.text] = upperWord if upperWord != 'DATE' else 'TIME'

    for token in doc:
        dep += [(token.i, token.text, token.lemma_, token.dep_, token.tag_, token.pos_, token.head.i, NERMapping[token.text])]

    return dep

def extractCambGP(en):
    dep = sentence_to_dependencyNER(cleanCambridgeSent(en))
    GPs = dependency_to_GP(dep, getRoot(dep))
    return GPs

In [3]:
cambridgeDict = eval(open('camDict.txt', 'r').read())

In [ ]:
from collections import defaultdict
import json

mapping = defaultdict(lambda: defaultdict(lambda: []))

index = 0
# cambridge
for word in cambridgeDict.keys():
    for pos in cambridgeDict[word].keys():
        for pair in cambridgeDict[word][pos]:
            if len(pair) == 3:
                en, ch, phrase = pair
                if en == '': continue
                GPs = extractCambGP(en)
                GPs = [ GP for GP in GPs if not (GP[2] == 'V adv' and GP[3] in unWantADV) ]
                mapping[word][pos].append((index, list(GPs)))

                file.write(str(index)+' ||| '+en+'\t'+ch+'\t'+phrase+'\n')
                index += 1 
            else:
                en, ch = pair
                if en == '': continue
                GPs = extractCambGP(en)
                GPs = [ GP for GP in GPs if not (GP[2] == 'V adv' and GP[3] in unWantADV) ]
                mapping[word][pos].append((index, list(GPs)))

                file.write(str(index)+' ||| '+en+'\t'+ch+'\n')
                index += 1 
                
file1.write(str(json.loads(json.dumps(mapping))))
file.close()
file1.close()